## Inferência

#### Teste estatístico de hipóteses

In [7]:
from typing import Tuple
import math

def normal_approximation_to_binomial(n: int, p: float) -> Tuple[float, float]:
    """Returns mu and sigma corresponding to a Binomial(n, p)"""
    
    mu = p * n
    sigma = math.sqrt(p * (1 - p) * n)
    
    return mu, sigma

In [8]:
def normal_cdf(x: float, mu: float = 0, sigma: float = 1) -> float:
    return (1 + math.erf((x - mu) / math.sqrt(2) / sigma)) / 2

# The normal cdf is the probability the variable is below a threshold
normal_probability_below = normal_cdf

In [4]:
# It's above the threshold if it's not below the threshold
def normal_probability_above(lo: float,
                             mu: float = 0,
                             sigma: float = 1) -> float:
    """The probability that a N (mu, sigma) is greater than lo."""
    
    return 1 - normal_cdf(lo, mu, sigma)

In [5]:
# It's between if it's less than hi, but not less than lo
def normal_probability_between(lo: float,
                               hi: float,
                               mu: float = 0,
                               sigma: float = 1) -> float:
    """The probability that N(mu, sigma) is between lo and hi"""
    
    return normal_cdf(hi, mu, sigma) - normal_cdf(lo, mu, sigma)

In [6]:
# It's outside if it's not between
def normal_probability_outside(lo: float,
                               hi: float,
                               mu: float = 0,
                               sigma: float = 1) -> float:
    """The probability that N(mu, sigma) is not between lo and hi"""
    
    return 1 - normal_probability_between(lo, hi, mu, sigma)

We can also do the opposite and find the region outside the threshold or the interval (symmetric) around the mean that is associated with a given level of probability.

In [18]:
def inversal_normal_cdf(p: float,
                        mu: float = 0,
                        sigma: float = 1,
                        tolerance: float = 0.00001) -> float:
    """Find approximate inverse using binary search"""
    
    if mu != 0 or sigma != 1:
        return mu + sigma * inversal_normal_cdf(p, tolerance=tolerance)
    
    low_z = -10.0          # normal_cdf(-10) is (very close to) 0
    hi_z = 10.0            # normal_cdf(10) is (very close to) 1
    
    while hi_z - low_z > tolerance:
        mid_z = (low_z + hi_z) / 2
        mid_p = normal_cdf(mid_z)
        
        if mid_p < p:
            low_z = mid_z
        else:
            hi_z = mid_z
    
    return mid_z

In [19]:
def normal_upper_bound(probability: float,
                       mu: float = 0,
                       sigma: float = 1) -> float:
    """Returns the z for which P(Z <= z) = probability"""
    
    return inversal_normal_cdf(probability, mu, sigma)

In [20]:
def normal_lower_bound(probability: float,
                       mu: float = 0,
                       sigma: float = 1) -> float:
    """Returns the z for which P(Z >= z) = probability"""
    
    return inversal_normal_cdf(1 - probability, mu, sigma)

In [21]:
def normal_two_sided_bounds(probability: float, 
                            mu: float = 0,
                            sigma: float = 1) -> Tuple[float, float]:
    """
    Returns the symetric (about the man) bounds that contain
    that especified probability
    """
    
    tail_probability = (1 - probability) / 2
    
    # upper bound should have tail_probability above it
    upper_bound = normal_lower_bound(tail_probability, mu, sigma)
    
    # lower bound should have tail_probability below it
    lower_bound = normal_upper_bound(tail_probability, mu, sigma)
    
    return lower_bound, upper_bound

In [22]:
mu_0, sigma_0 = normal_approximation_to_binomial(1000, 0.5)

assert mu_0 == 500
assert 15.8 < sigma_0 < 15.9

In [24]:
# significância (469, 531)

lower_bound, upper_bound = normal_two_sided_bounds(0.95, mu_0, sigma_0)

assert 468.5 < lower_bound < 469.5
assert 530.5 < upper_bound < 531.5

In [31]:
# p = 0.5

lo, hi = normal_two_sided_bounds(0.95, mu_0, sigma_0)

mu_1, sigma_1 = normal_approximation_to_binomial(1000, 0.55)

# um erro tipo 2 indica a rejeição da hipotese nula

type_2_probability = normal_probability_between(lo, hi, mu_1, sigma_1)

power = 1 - type_2_probability

assert 0.886 < power < 0.888

print(power)

0.8865480012953671


In [33]:
hi = normal_upper_bound(0.95, mu_0, sigma_0)

type_2_probability = normal_probability_below(hi, mu_1, sigma_1)

power = 1 - type_2_probability 

assert 526 < hi < 526.1
assert 0.9363 < power < 0.9364

print(hi, power)

526.0073585242053 0.9363794803307173
